In [1]:
import csv,re, requests, os
from bs4 import BeautifulSoup

url = 'https://www.truemodel.com/results/women-main/women'
basedir = 'truemodel.com'
models_details = list()

In [2]:
soup = BeautifulSoup(requests.get(url).content, 'html5lib')

In [3]:
urls = [url['href'] for url in soup.find('ul', class_='dropdown-menu').findAll('a')]
soups = [BeautifulSoup(requests.get(url).content, 'html5lib') for url in urls]

model_urls = [link['href'] for soup in soups for link in soup.find('div', class_='results-grid').findAll('a')]
model_names = [name.find('div', class_='name').text.strip().replace(' ','').replace('\n',' ') for soup in soups for name in soup.find('div', class_='results-grid').findAll('a')]


In [4]:

models_dict = dict(zip(model_names, model_urls))
models_dict.pop('TheGozaTwins')


'https://www.truemodel.com/portfolio/women-main/women/1995345/the-goza-twins'

In [5]:
len(models_dict)

147

In [14]:
def parse(soup: BeautifulSoup) -> dict:
    features = [feat.text.upper() for feat in soup.find('div', class_='stats').findAll('span', class_='stat-label')]
    value = [val.text.strip() for val in soup.find('div', class_='stats').findAll('span', class_='value')]
    return dict(zip(features,value))
def download_pics(pics):
    patt=r"\d+.jpg"
    for pic in pics:
        regex = re.search(patt, pic).group(0)
        path = f"{basedir}/{k}/{regex}"
        if not os.path.exists(path):
            with open(path, 'wb') as f:
                f.write(requests.get(pic).content)
def list_to_csv(lst):
    with open(f"{basedir}/models_info.csv",'w') as f:
        c = csv.DictWriter(f, ["NAME","URL", "HEIGHT", "BUST","WAIST","HIPS","DRESS" ,"EYES","HAIR", "SHOE"])
        c.writeheader()
        for row in lst:
            c.writerow(row)

In [12]:
for k,v in models_dict.items():
    os.makedirs(f"{basedir}/{k}",exist_ok=True)
    soup=BeautifulSoup(requests.get(v).content, 'html5lib')
    pics = [pic['data-thumb'] for pic in soup.find('div', {'class': 'row'}).findAll('a') ]
    download_pics(pics)
    feat_dict = {'NAME':k, "URL":v}
    feat_dict.update(parse(soup))
    models_details.append(feat_dict)


In [13]:
list_to_csv(models_details)